In [1]:
%scala

// Import Necessary Libraries
import org.joda.time._
import org.joda.time.format._
import com.microsoft.azure.cosmosdb.spark.schema._
import com.microsoft.azure.cosmosdb.spark._
import com.microsoft.azure.cosmosdb.spark.config.Config

// Maps
val readConfigMap = Map(
"Endpoint" -> "https://cdbtwitter.documents.azure.com:443/",
"Masterkey" -> "AnfVszwZbztwIPLheHScaZXoFa7kbWSoYYf75wttJ5Z502xEbeY6RlwHshRuntMisX5bOUzwKa6JvEX00MJi2w==",
"Database" -> "Twitterdb",
"Collection" -> "Twittercoll", 
"preferredRegions" -> "East US",
"SamplingRatio" -> "1.0",
"schema_samplesize" -> "200000",
"query_custom" -> "SELECT c.id, c.created_at, c.user.screen_name, c.user.location, c.text, c.retweet_count, c.entities.hashtags, c.entities.user_mentions, c.favorited, c.source FROM c"
)

val writeConfigMap = Map(
"Endpoint" -> "https://cdbtwitter.documents.azure.com:443/",
"Masterkey" -> "AnfVszwZbztwIPLheHScaZXoFa7kbWSoYYf75wttJ5Z502xEbeY6RlwHshRuntMisX5bOUzwKa6JvEX00MJi2w==",
"Database" -> "Twitterdb",
"Collection" -> "Twittercoll1",  
"preferredRegions" -> "East US",
"SamplingRatio" -> "1.0",
"schema_samplesize" -> "200000"
)

// Configs
// get read
val readConfig = Config(readConfigMap)
val tweets = spark.read.cosmosDB(readConfig)
tweets.createOrReplaceTempView("tweets")
tweets.cache()

// get write
val writeConfig = Config(writeConfigMap)

import org.joda.time._
import org.joda.time.format._
import com.microsoft.azure.cosmosdb.spark.schema._
import com.microsoft.azure.cosmosdb.spark._
import com.microsoft.azure.cosmosdb.spark.config.Config
readConfigMap: scala.collection.immutable.Map[String,String] = Map(query_custom -> SELECT c.id, c.created_at, c.user.screen_name, c.user.location, c.text, c.retweet_count, c.entities.hashtags, c.entities.user_mentions, c.favorited, c.source FROM c, Collection -> Twittercoll, Endpoint -> https://cdbtwitter.documents.azure.com:443/, Database -> Twitterdb, SamplingRatio -> 1.0, schema_samplesize -> 200000, preferredRegions -> East US, Masterkey -> AnfVszwZbztwIPLheHScaZXoFa7kbWSoYYf75wttJ5Z502xEbeY6RlwHshRuntMisX5bOUzwKa6JvEX00MJi2w==)
writeConfigMap: scala.collection.immutable.Map[String,String] = Map(Collection -> Twittercoll1, Endpoint -> https://cdbtwitter.documents.azure.com:443/, Database -> Twitterdb, SamplingRatio -> 1.0, schema_samplesize -> 200000, preferredRegions -> East US, Masterkey -> AnfVszwZbztwIPLheHScaZXoFa7kbWSoYYf75wttJ5Z502xEbeY6RlwHshRuntMisX5bOUzwKa6JvEX00MJi2w==)
readConfig: com.microsoft.azure.cosmosdb.spark.config.Config = com.microsoft.azure.cosmosdb.spark.config.ConfigBuilder$$anon$1@e8c4049a
tweets: org.apache.spark.sql.DataFrame = [created_at: string, favorited: boolean ... 8 more fields]
writeConfig: com.microsoft.azure.cosmosdb.spark.config.Config = com.microsoft.azure.cosmosdb.spark.config.ConfigBuilder$$anon$1@5e83d2a0

In [2]:
%sql
select hashtags.text, count(distinct id) as tweets
from (
select 
  explode(hashtags) as hashtags,
  id
from tweets
) a
group by hashtags.text
order by tweets desc
limit 10

text,tweets
AI,283
BigData,203
MachineLearning,198
DeepLearning,132
IoT,114
DataScience,101
ArtificialIntelligence,94
ML,76
DL,57
Infographics,56


In [3]:
%scala
// Import SaveMode so you can Overwrite, Append, ErrorIfExists, Ignore
import org.apache.spark.sql.{Row, SaveMode, SparkSession}

// Create new DataFrame of tweets tags
val tweets_bytags = spark.sql("select '2018-06-12' as currdt, hashtags.text as hashtags, count(distinct id) as tweets from ( select explode(hashtags) as hashtags, id from tweets ) a group by hashtags.text order by tweets desc limit 10")

// Save to Cosmos DB (using Append in this case)
//    Ensure the baseConfig contains a Read-Write Key
//    The key provided in our examples is a Read-Only Key
//tweets_bytags.write.mode(SaveMode.Append).cosmosDB(writeConfig)
tweets_bytags.write.mode(SaveMode.Overwrite).cosmosDB(writeConfig)

import org.apache.spark.sql.{Row, SaveMode, SparkSession}
tweets_bytags: org.apache.spark.sql.DataFrame = [currdt: string, hashtags: string ... 1 more field]